# Lab 3: Busquedas de texto autocompletadas

Una funcionalidad muy demandada a la hora de hacer burscadores para aplicaciones el la búsqueda atuocompletada. Esta consiste en ir sugiriendo al usuario como completar el término de búsqueda según lo va escribiendo el en el buscador en función de los valores que encontramos en la base de datos.

<img src="../../images/els/els41.png" alt="Busqueda autocompletada"/>

En este notebook vamos a ver las distintas altenativas que tenemos para realizar esta funcionalidad:

* Prefix Query, Phrase Prefix Query
* Completion suggest
* Analizador n-grams

Para realizar los ejercicios propuestos en este notebook, puedes utilizar el Dev Tools de Kibana. Puedes acceder a él a través de este enlace: http://127.0.0.1:5601

# 1. Prefix Query, Phrase Prefix Query

Haciendo un recordatorio de lo que hemos visto previamente, podemos declarar de dos formas diferentes los campos de tipo texto:

* **text ⇒ analyzed text**
* **keyword ⇒ not analyzed text**


1. Quizás es buen momento de que vuelvas a repasar el mapping type del índice del ejercicio que hemos creado antes para identificar que tipo a asignado Elasticsearch a los valores de tipo texto.

## 1.1. Prefix Query

La primera opción que tenemos para crear una búsqueda autocompletada es utilizar las Prefix Query. Estas queries se realizan sobre campos de tipo **keyword**, por lo que no ser realiza sobre campos analizados. Comprueba que el campo especificado cumple con el prefijo indicado en la query:

`
GET recipes/_search
{
  "query": { "prefix": { "author": "El" } },
  "_source": [ "author" ]
}
`

1. ¿Qué pasa si en vez de indicar 'El' se indica 'el' en la query?
2. ¿Y si en vez de 'El' se indica 'Eli'?

Como vemos este tipo de consultas es muy limitado ya que sólo comprueba el prefijo.

En el contexto de las búsquedas autocompletadas, para mejorar las sugerencias, es muy común comprobar las distintas combinaciones de las letras del termino de búsqueda dentro de las palabras de los campos a buscar para sugerir respuestas. Por ejemplo, si buscamos Eli, puede ser interesante que se sugiera como término Elisabeth y Elvis. Como podemos ver Elvis no empieza pro Eli, pero si tiene una i a continuación e El. Es muy común sugerir estos resultados para evitar los typo error (errores de escritura, ordenes le las letras, letras que faltan...)

Una forma de incorporar este comportamiento a las Prefix Query es combinarlas con las Fuzzi Queries. 

Las fuzzi queries buscan similaridad entre las palabras, así una *fuzzi query* con nivel de *fizziness* 1 devolvería *Mario* y *Dario* sobre una búsqueda del término *Mario*.

Vamos a ver un ejemplo:

`
GET recipes/_search
{
  "query": {
    "bool": {
      "should": [
        { "prefix": { "author": "Elia" } },
        { "fuzzy": { "author": { "value": "Elia", "fuzziness": 2, "prefix_length": 0 } } }
      ]
    }
  },
  "_source": [ "author" ]
}
`

1. ¿Qué resultdao obtenemos si en vez de buscar por 'Eli' buscamos por 'Elia'?


## 1.2. Match Phrase Prefix Query

Las Match Phrase Prefix Query se realizan sobre campos de tipo **text**, es decir sobre campos **analizados**. Por esta razón permite hacer búsquedas sobre todas las palabras que se encuentren en una frase buscando prefijos y no sólo sobre el prefijo de la primera palabra de la frase.

`
GET recipes/_search
{
  "query": {
    "match_phrase_prefix": {
      "author.text": {
        "query": "De",
        "max_expansions": 10
      }
    }
  },
  "_source": ["author"]
}
`

1. ¿En qué lugar se encuentra el prefijo buscado en la frase del resultado devuelto?
2. Realiza la buscqueda con el emismo término (De) con una prefix query. ¿Que diferencia hay con respecto de la anterior?

`
GET recipes/_search
{
  "query": { "prefix": { "author": "De" } },
  "_source": [ "author" ]
}
`

Estas sentencias no soportan *fuzziness*.


## 1.3 Conclusiones

* Son sencillas de implementar.
* El rendimiento disminuye según van creciendo los documentos indexados en el índice. Buenas para datasets pequeños.
* Necesitan combinarse con *Fuzzi queries*


# 2. Completion Suggester

Es el mecanismo que provee Elasticserach para hacer este tipo de búsquedas 'type as you go'.

* Ofrece sugerencias de búsqueda sobre los elementos almacenados en un campo concreto. 
* Están pensados para que se ejecuten de forma rápida, por lo que normalmente construyen un índice en memoria que acelere su ejecución.
* Sólo trabajan sobre prefijos y expresiones regulares.
* Es case insensitive.
* Es necesario indicar en el mapping type que los campos por los que queremos realizar este tipo de búsqueda son de tipo completion.

Si consultamos el mapping del índice de recipies y nos fijamos en el campo title veremos que se ha definido con un multi-filed, donde hemos indicado que el campo title ademas se indexe en un campo de tipo "completion" que nos permite realizar consulta de tipo suggest sobre él para realizar búsquedas autocompletadas:

* preserve_separators: mantiene los separadores (por defecto true). Si se deshabilita un campo que contenga el texto "Foo Fighters" machearia con la búsqueda 'foof'.
* preserve_position_increments: si se deshabilita y se usa un analizador con stop words, si buscamos el texto 'b' sobre un campo que contenga "The Beatles" macheará el resultado.

`
"title": {
  "type": "text",
  "analyzer": "english",
  "fields": {
    "keyword": {
      "type": "keyword"
    },
    "suggestion": {
      "type": "completion",
      "analyzer": "english",
      "preserve_separators": false,
      "preserve_position_increments": false,
      "max_input_length": 50
    },
    "ngram": {
        "type": "text",
        "analyzer": "autocomplete",
        "search_analyzer": "autocomplete_search"
    }
  }
}
`

Vamos a utilizar el suggest sobre el campo title para ver que nos sugiere si escribimos 'Ble'

`
POST recipes/_search?pretty
{
    "suggest": {
        "recipies-suggest" : {
            "prefix" : "Ble", 
            "completion" : { 
                "field" : "title.suggestion" 
            }
        }
    },
    "_source": ["title"]
}
`

1. ¿Qué pasa si buscamos el texo 'May'? ¿Encuentra el documento con el título "Blender Mayonnaise"? 

## 2.1. Conclusiones

* Más rápidas que las Prefix Query, Phrase Prefix Query, se realizan en near real time, idealmente tan rápido como el usuario escribre el téxto de búsqueda.
* Necesitan más memoria para ejecutarse.
* La búsqueda siempre se realiza sobre el principio de la frase.

# 3. N-Gram Analyzer

Antes de ver como se utiliza el N-Gram analyzer para implementar este tipo de búsquedas, vamos a pararnos en estudiar el análisis que realiza el n-gram tokenizer que vamos a utilizar para indexar los documentos.

* El nGram_tokenizer se encarga de generar todos los substring (n-grams) que contiene una frase.
* Cada n-gram es una entrada en el índice inverso.

Vamos a ver como funciona, ejecuta la siguiente sentencia en Kibana:

`
GET _analyze
{
  "tokenizer": "ngram",
  "text": ["Blender Chermoula Sauce"]
}
`

El resultado es el conjunto de n-grams que extrae del texto que le pasamaos. 

Como podemos deducir del resultado del tokenizer, este análisis nos va a permitir realizar búsquedas sobre todos los términos de una frase, independientemente de donde se encuentren, al principio de la frase, como prefijo de una palabra, o contenido dentro de una palabra.

Para utilizar el tokenizer tenemos que definirlo al crear el mapping type dentro del apartado "analysis". Al configurar el nGram tokenizer se puede especificar la longitud máxima y mínima que pueden tener las substring por medio de los parámetros min_gram y max_gram.

Una vez definido el tokenizer, tenemos que definir un analizador que lo utilice y esto se hace igualmente al crear el mapping type del índice. Por último asociáremos el analizador a los campos por los que vamos a realizar la búsqueda. Puesto que son campos que vamos a analizar tendrán que ser de tipo **text**.


Si consultamos nuevamente elmapping del índice "recipies" vermos que se ha difino un analizador que utiliza el tokenizador de tipo ngram con nombre "autocomplete".
Este analizador se está utilizando en el campo "title" en el multifield "ngram" para que el campo title además se analice con este analizador para poder realizar las búsquedas autocompletadas sobre él:

`
"analysis": {
    "analyzer": {
        "autocomplete": {
          "tokenizer": "autocomplete",
          "filter": [
            "lowercase"
          ]
        },
        "autocomplete_search": {
          "tokenizer": "lowercase"
        }
    },
    "tokenizer": {
        "autocomplete": {
          "type": "ngram",
          "min_gram": 1,
          "max_gram": 2,
          "token_chars": [
            "letter"
          ]
        }
    }
} 
`

`
"mappings": {
    "properties": {
      "title": {
          "type": "text",
          "analyzer": "english",
          "fields": {
            "keyword": {
              "type": "keyword"
            },
            "suggestion": {
              "type": "completion",
              "analyzer": "english",
              "preserve_separators": false,
              "preserve_position_increments": false,
              "max_input_length": 50
            },
            "ngram": {
                "type": "text",
                "analyzer": "autocomplete",
                "search_analyzer": "autocomplete_search"
            }
          }
        }
    }
}
`

1. ¿Para qué sirve el search_analyzer? Para saber más entra en este enlace: https://www.elastic.co/guide/en/elasticsearch/reference/current/search-analyzer.html
2. ¿Para qué definimos el filtro lowercase?

Comprobemos como funciona el analizador que hemos definido:

`
POST recipes/_analyze
{
  "analyzer": "autocomplete",
  "text": "Blender Chermoula Sauce"
}
`

El resultado obtenido son todos los terminos que extrae de la frase que le hemos indicado.

Veamos como funcionan las bśuquedas:

`
GET recipes/_search
{
  "query": {
    "match": {
      "title.ngram": {
        "query": "ble"
      }
    }
  },
    "_source": ["title"]
}
`

1. Comparalo con las búsquedas con el campo completion. 

Es muy común que a la hora de implementar este tipo de búsqueda queramos lanzarla sobre varios campos de un documento, por ejemplo sobre el título de una receta, el resumen de la receta, el autor, etc.

Podemos utilizar las búsquedas de tipo query_string para buscar sobre todos los campos de tipo text y keyword a la vez:

`
GET recipes/_search
{
    "query": {
        "query_string" : {
            "query" : "Ble"
        }
    }
}
`

Con el campo fields podemos especificar sobre que campos en concreto buscar
`
GET recipes/_search
{
    "query": {
        "query_string" : {
            "query" : "ble",
            "fields"  : ["title", "author", "author.text", "title.keyword", "title.ngram"]
        }
    }
}
`



## 3.1 Conclusiones

* Lo utilizaremos si necesitamos hacer búsquedas por cualquier término de un campo en cualquier orden.
* El tiempo de indexado es mayor, ya que el análisis que realiza es mas costoso.
* Ocupa más espacio en disco, ya que al extraer todos lo ngrams de los textos las entradas en los índices crecen.
